<a href="https://colab.research.google.com/github/JuanPabloSolano1/DR/blob/master/SQL_Script_Provider_Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
WITH
  pt
    AS (
      SELECT *
      FROM (
        SELECT
          distinct(customer_id) as customer_id,
          CURRENT_DATE() AS Today_date,
          MAX(date_created) AS day_last_lead,
          DATE_DIFF(
            CURRENT_DATE(
            ), CAST(MAX(date_created) AS date), DAY) as last_lead_received,
          CASE
            WHEN
                DATE_DIFF(
                  CURRENT_DATE(
                  ), CAST(MAX(date_created) AS date), DAY) <= 28
              THEN '28 Days'
            WHEN
                DATE_DIFF(
                  CURRENT_DATE(
                  ), CAST(MAX(date_created) AS date), DAY) BETWEEN 29
                and 91
              THEN '91 Days'
            WHEN
                DATE_DIFF(
                  CURRENT_DATE(
                  ), CAST(MAX(date_created) AS date), DAY) BETWEEN 91
                and 182
              THEN '182 Days'
            WHEN
                DATE_DIFF(CURRENT_DATE(), CAST(MAX(date_created) AS date), DAY)
                > 182
              THEN '+182 Days'
            ELSE '0'
          END as time_frame,
          IF(
            DATE_DIFF(
              CURRENT_DATE(
              ), CAST(MAX(date_created) AS date), DAY) <= 28,
            1,
            0
          ) AS check,
          IF(
            DATE_DIFF(
              CURRENT_DATE(
              ), CAST(MAX(date_created) AS date), DAY) BETWEEN 29
            and 91,
            1,
            0
          ) check_2,
          IF(
            DATE_DIFF(
              CURRENT_DATE(
              ), CAST(MAX(date_created) AS date), DAY) BETWEEN 91
            and 182,
            1,
            0
          ) check_3,
          IF(
            DATE_DIFF(CURRENT_DATE(), CAST(MAX(date_created) AS date), DAY)
            > 182,
            DATE_DIFF(CURRENT_DATE(), CAST(MAX(date_created) AS date), DAY),
            0
          ),
          COUNT(
            DISTINCT(IF(charge_status = 'charged', lead_id, null))
          ) as total_charged_leads,
          SUM(lead_charge_usd) as lead_charge_status
        FROM ads_homeservices.ghs_account_management_leads a
        GROUP BY 1
      )
    ),
    roster
      AS (
        SELECT
          t1.customer_id,
          t1.business_name as business_name,
          scr.region as region,
          t1.CategoryString as category,
          t1.churn_status as churn_status
        FROM ads_homeservices.ghs_roster
        WHERE t1.customer_id IS NOT NULL
      ),
      coverage
        AS (
          SELECT
            t1.customer_id,
            scr.region as c_region,
            t1.CategoryString as c_category,
            t1.churn_status as c_churn_status,
            t4.date_of_first_impression as first_impression,
            DATE_DIFF(
              CURRENT_DATE, CAST(t4.date_of_first_impression AS DATE), DAY
            ) as days_live,
            COUNT(
              CONCAT(
                scr.region, t1.CategoryString, CAST(t1.churn_status AS STRING)
              )
            ) as coverage_1
          FROM ads_homeservices.ghs_roster
          GROUP BY 1, 2, 3, 4, 5
          ORDER BY
            CONCAT(
              scr.region, t1.CategoryString, CAST(t1.churn_status AS STRING)
            )
        ),
        c
          AS (
            SELECT customer_id, COUNT(zip_code) as total_zip_codes
            FROM ads_homeservices.gls_pro_zip
            GROUP BY 1
          ),
          table_4
            AS (
              SELECT
                customer_id,
                COUNT(
                  DISTINCT(IF(charge_status = 'charged', lead_id, null))
                ) as charged_28_days,
                SUM(lead_charge_usd) AS lead_charge_usd_28
              FROM ads_homeservices.ghs_account_management_leads
              WHERE
                DATE_DIFF(CURRENT_DATE(), CAST(date_created AS DATE), DAY) <= 28
              GROUP BY 1
            ),
            table_5
              AS (
                SELECT
                  customer_id,
                  COUNT(DISTINCT(
                    IF(charge_status = 'charged', lead_id, null))
                  ) as charged_leads_28_days_after_first_impression,
                 SUM(lead_charge_usd) as lead_charge_usd_28_days_after_first_impression
                FROM
                    ads_homeservices.ghs_account_management_leads
                  LEFT JOIN ads_homeservices.ghs_roster
                  ON t1.customer_id = customer_id
                WHERE
                  date_created BETWEEN CAST(t4.date_of_first_impression AS DATE)
                  AND DATE_ADD(
                    CAST(t4.date_of_first_impression AS DATE), INTERVAL 28 DAY
                  )
                GROUP BY 1
              )
                SELECT
                  DISTINCT(roster.customer_id),
                  roster
                  .business_name,
                    c_region,
                    category,
                    first_impression,
                    days_live,
                    Today_date,
                    day_last_lead,
                    last_lead_received,
                    time_frame,
                    check,
                    check_2,
                    check_3,
                    c_churn_status,
                    total_zip_codes,
                    IFNULL(charged_28_days, 0) as charged_28_days,
                    total_charged_leads,
                    churn_status,
                    ROUND(lead_charge_status, 0) AS total_charge_status,
                    IFNULL(ROUND(lead_charge_usd_28, 0), 0) lead_charge_usd_28,
                    IFNULL(
                      charged_leads_28_days_after_first_impression, 0
                    ) AS charged_leads_28_days_after_first_impression,
                    IFNULL(ROUND(lead_charge_usd_28_days_after_first_impression,0),0) AS lead_charge_usd_28_days_after_first_impression,
                    IF(time_frame = '28 Days' , 1 , 0) as count_providers_with_leads
                    
                from
                    pt
                  LEFT JOIN coverage ON pt.customer_id = coverage.customer_id
                  LEFT JOIN c ON c.customer_id = pt.customer_id
                  LEFT JOIN roster
                  ON
                    CONCAT(
                      c_region, c_category
                    ) = CONCAT(roster.region, roster.category)
                    AND pt.customer_id = roster.customer_id
                  LEFT JOIN table_4 ON pt.customer_id = table_4.customer_id
                  LEFT JOIN table_5 ON pt.customer_id = table_5.customer_id
                WHERE roster.customer_id IS NOT NULL
                GROUP BY
                  1,
                  2,
                  3,
                  4,
                  5,
                  6,
                  7,
                  8,
                  9,
                  10,
                  11,
                  12,
                  13,
                  14,
                  15,
                  16,
                  17,
                  18,
                  19,
                  20,
                  21,
                  22,
                  23
                ORDER BY time_frame DESC
#limit 10
